In [6]:
%load_ext autoreload
%autoreload 2

import sys 
import os
sys.path.append('/home/azhar04/project/1. dev/quantum-dev/ShotOptimized-ADAPT-VQE/')

from openfermion.transforms import jordan_wigner
from openfermion.utils import commutator

from src.pools import SD, GSD, GSD1, SingletGSD, SpinCompGSD, PauliPool,  NoZPauliPool1, NoZPauliPool, QE, QE1, QE_All, CEO, OVP_CEO, DVG_CEO, DVE_CEO, MVP_CEO
from src.molecules import create_h2

# Molecule
R = 0.742
molecule = create_h2(R)
molecule.description

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'H2'

In [5]:
H = molecule.get_molecular_hamiltonian()
Hq = jordan_wigner(H)
Hq

-0.0996319255760924 [] +
-0.04533060139114829 [X0 X1 Y2 Y3] +
0.04533060139114829 [X0 Y1 Y2 X3] +
0.04533060139114829 [Y0 X1 X2 Y3] +
-0.04533060139114829 [Y0 Y1 X2 X3] +
0.17110568429296397 [Z0] +
0.1685935681755208 [Z0 Z1] +
0.1205103673498828 [Z0 Z2] +
0.1658409687410311 [Z0 Z3] +
0.17110568429296397 [Z1] +
0.1658409687410311 [Z1 Z2] +
0.1205103673498828 [Z1 Z3] +
-0.22250984629381015 [Z2] +
0.1743208433444256 [Z2 Z3] +
-0.22250984629381015 [Z3]

Qubit Commutator

In [ ]:
qubit_pool = NoZPauliPool(molecule)

for k in range(len(qubit_pool.operators)):
    Ak = qubit_pool.operators[k]._q_operator
    print(f'\nOperator-{k}: {Ak}')
    
    grad_obs = commutator(Hq, Ak)
    print(f'Grad Obs-{k}: {grad_obs}')
    # print(f'\nOperator-{i}: {type(qubit_pool.operators[i]._f_operator)}')


Operator-0: 1j [Y0 X2]
Grad Obs-0: (0.3371871363510416-0j) [X0 Z1 X2] +
(0.34221136858592793-0j) [X0 X2] +
(0.3316819374820622-0j) [X0 X2 Z3] +
(-0.3316819374820622+0j) [Y0 Z1 Y2] +
(0.4450196925876203-0j) [Y0 Y2] +
(-0.3486416866888512+0j) [Y0 Y2 Z3]

Operator-1: 1j [X0 Y2]
Grad Obs-1: (0.3316819374820622-0j) [X0 Z1 X2] +
(-0.4450196925876203+0j) [X0 X2] +
(0.3486416866888512-0j) [X0 X2 Z3] +
(-0.3371871363510416+0j) [Y0 Z1 Y2] +
(-0.34221136858592793+0j) [Y0 Y2] +
(-0.3316819374820622+0j) [Y0 Y2 Z3]

Operator-2: 1j [Y1 X3]
Grad Obs-2: (0.3371871363510416-0j) [Z0 X1 X3] +
(-0.3316819374820622+0j) [Z0 Y1 Y3] +
(0.3316819374820622-0j) [X1 Z2 X3] +
(0.34221136858592793-0j) [X1 X3] +
(-0.3486416866888512+0j) [Y1 Z2 Y3] +
(0.4450196925876203-0j) [Y1 Y3]

Operator-3: 1j [X1 Y3]
Grad Obs-3: (0.3316819374820622-0j) [Z0 X1 X3] +
(-0.3371871363510416+0j) [Z0 Y1 Y3] +
(0.3486416866888512-0j) [X1 Z2 X3] +
(-0.4450196925876203+0j) [X1 X3] +
(-0.3316819374820622+0j) [Y1 Z2 Y3] +
(-0.34221136858592